This program takes in: 
1) a list of opted-in emails  
2) the participants file from the previous round  
3) the master list of participants

and returns 
1) the participants file for the upcoming round  

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox
from IPython.display import HTML
import base64
import time

In [3]:
style = {'description_width': 'auto'}

master_widget = widgets.Text(
    description='Master file:', 
    disabled=False, 
    style=style)

file_widget = widgets.Text(
    description='Participants file:', 
    disabled=False, 
    style=style)

optin_widget = widgets.Text(
    description='Opt-in file:', 
    disabled=False, 
    style=style)

datafile_widget = widgets.Text(
    description='Data file:', 
    disabled=False, 
    style=style)

round_widget = widgets.Dropdown(
    options=np.arange(1, 10),
    description='Matching round:', 
    disabled=False, 
    style=style)

import_button = widgets.Button(description='Import', style=style)
sync_button = widgets.Button(description='Sync', style=style)
download_data_button = widgets.Button(description='Download Data', style=style)

def display_widget():
    display(round_widget), \
    display(master_widget), \
    display(file_widget), \
    display(optin_widget), \
    display(import_button), \
    display(sync_button), \
    display(download_data_button)

In [4]:
def event_import(button):
    global master_data, participants_data, optin
    # imports the participants data and opt-in email list
    clear_output()
    display_widget()
        
    master_data = pd.read_excel(master_widget.value, index_col=0)
    participants_data = pd.read_excel(file_widget.value, index_col=0)
    optin = pd.read_excel(optin_widget.value, names=['email'])
    
    # make lowercase and remove spaces
    master_data['email'] = master_data['email'].str.lower()
    master_data['email'] = master_data['email'].str.strip()
    participants_data['email'] = participants_data['email'].str.lower()
    participants_data['email'] = participants_data['email'].str.strip()
    optin['email'] = optin['email'].str.lower()
    optin['email'] = optin['email'].str.strip()
    
    print('Imported')

In [5]:
def event_sync(button):
    # match up the two datasets 
    global master_data, participants_data, optin
    clear_output()
    display_widget()
    
    # populate the participants list with recent additions to the master list
    participants_data = pd.merge(master_data, participants_data, how='outer')
    
    # check if the length of the intersection between the two lists is the same as the length of optin
    # if it isn't, then that means there is an error
    if np.count_nonzero(np.in1d(participants_data['email'].to_numpy(), optin['email'].to_numpy())) != \
                len(optin):
        if np.count_nonzero(np.in1d(participants_data['email'].to_numpy(), optin['email'].to_numpy())) < \
            len(optin):
            print('Lengths don\'t match, likely an opt-in participant didn\'t fill out the intake form')
            print('Fix the participants / opt-in lists and start again')
    
        elif np.count_nonzero(np.in1d(participants_data['email'].to_numpy(), optin['email'].to_numpy())) > \
            len(optin):
            print('Lengths don\'t match, likely there is a duplicate in the intake form')
            print('Fix the participants / opt-in lists and start again')
    
    else:
        print('Lengths match, good, proceeding')
        time.sleep(2)
        # populate the current round opt-in column from the opt-in list
        participants_data['round'+str(round_widget.value)] = np.zeros(len(participants_data))
        for i, answer in enumerate(np.in1d(participants_data['email'].to_numpy(), \
                                           optin['email'].to_numpy())):
            if answer:
                participants_data.loc[i, 'round'+str(round_widget.value)] = 'Yes'
            else:
                participants_data.loc[i, 'round'+str(round_widget.value)] = 'No'
        
        # replace field that's entirely space (or empty) with 'No'
        participants_data.replace(r'^\s*$', 'No', regex=True)
                    
        if np.any(participants_data['round'+str(round_widget.value)]) == 0:
            # make sure they were each given 'Yes' or 'No'
            print('ERROR')
        else:
            print('All done')

In [6]:
def create_download_link(df, title="Download CSV file", filename="data.csv"): 
    # turns the pandas DataFrame into a csv to download
    csv = df.to_csv(index=True)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    display(HTML(html))
    
def event_download_data(button):
    # button to download the participants data
    global master_data, participants_data, optin
    clear_output()
    display_widget()
    return create_download_link(participants_data, \
                                title="download round %i participants"%round_widget.value, \
                                filename="round_%i_participants.csv"%round_widget.value)

In [7]:
# connecting the jupyter buttons to the actions for each button 
import_button.on_click(event_import)
sync_button.on_click(event_sync)
download_data_button.on_click(event_download_data)

In [8]:
clear_output()
display_widget()

Dropdown(description='Matching round:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9), style=DescriptionStyle(descriptio…

Text(value='Spring2023_program/round_1_master.xlsx', description='Master file:', style=TextStyle(description_w…

Text(value='Spring2023_program/round_1_master.xlsx', description='Participants file:', style=TextStyle(descrip…

Text(value='Spring2023_program/round_1_optin.xlsx', description='Opt-in file:', style=TextStyle(description_wi…

Button(description='Import', style=ButtonStyle())

Button(description='Sync', style=ButtonStyle())

Button(description='Download Data', style=ButtonStyle())